# Install Package

In [ ]:
# 아래 환경 설치 후 반드시 런타임 재시작 할것
!pip install transformers
!pip install tensorflow_hub

# Import Module

In [1]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
import transformers
import tensorflow_hub as hub
from transformers import GPT2Tokenizer, TFGPT2Model
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Input, Dropout, Lambda
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
df = pd.read_csv('Petition_data_english.csv')
df = df.loc[:, ['content_eng', 'label']]
df.head()

,content_eng,label
0,The development of petition AI image generator...,1
1,A petition for severe punishment and disclosur...,1
2,Do you know that the petition government on th...,1
3,KEPCO's petition against privatization of publ...,1
4,I entered OO Elementary School in 2003 and tra...,1


# Data Preprocessing

In [3]:
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()
    
    # 특수문자 제거
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    # 숫자 제거
    text = re.sub('\w*\d\w*', '', text)
    
    # 불용어 처리
    words = text.split()
    words = [word for word in words if not word in stop_words]
    text = " ".join(words)
    
    return text
  
df['content_eng'] = df['content_eng'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
texts = df["content_eng"].tolist()
labels = df["label"].tolist()

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenized_texts = [tokenizer.encode(text, add_special_tokens=True) for text in texts]
max_length = max(len(text) for text in tokenized_texts)
padded_texts = tf.keras.preprocessing.sequence.pad_sequences(tokenized_texts, maxlen=max_length, padding='post', truncating='post')

#### Train, Test 분리

In [6]:
# 훈련, 검증 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_texts, labels, test_size=0.2, random_state=42)

In [7]:
# 레이블 크기 변경
y_train = np.array(y_train).reshape(-1, 1)
y_test = np.array(y_test).reshape(-1, 1)

# Modeling

In [8]:
# 모델링
gpt_model = TFGPT2Model.from_pretrained("gpt2")
gpt_model.trainable = False # 미세조정하지 않음

input_layer = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32)
gpt_output = gpt_model(input_layer)[0]
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(gpt_output)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

checkpoint = ModelCheckpoint('GPT_eng.h5',
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             mode='min',
                             verbose=False)

# 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Epoch 1/50
14/14 [==============================] - 37s 2s/step - loss: 2.9873 - accuracy: 0.7162 - val_loss: 2.3322 - val_accuracy: 0.7897
Epoch 2/50
14/14 [==============================] - 21s 1s/step - loss: 1.9273 - accuracy: 0.7397 - val_loss: 2.3537 - val_accuracy: 0.5433
Epoch 3/50
14/14 [==============================] - 22s 2s/step - loss: 1.5120 - accuracy: 0.7651 - val_loss: 1.4584 - val_accuracy: 0.6582
Epoch 4/50
14/14 [==============================] - 23s 2s/step - loss: 1.2092 - accuracy: 0.7911 - val_loss: 1.0573 - val_accuracy: 0.6827
Epoch 5/50
14/14 [==============================] - 23s 2s/step - loss: 0.9478 - accuracy: 0.7651 - val_loss: 0.7264 - val_accuracy: 0.7723
Epoch 6/50
14/14 [==============================] - 23s 2s/step - loss: 0.7809 - accuracy: 0.7663 - val_loss: 0.6324 - val_accuracy: 0.7709
Epoch 7/50
14/14 [==============================] - 22s 2s/step - loss: 0.7084 - accuracy: 0.7552 - val_loss: 0.6642 - val_accuracy: 0.7775
Epoch 8/50
14/14 [==

In [9]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('GPT_eng.h5')
model.evaluate(X_test, y_test)

4/4 [==============================] - 4s 945ms/step - loss: 0.4939 - accuracy: 0.7880


[0.49389058351516724, 0.7879510521888733]

In [10]:
# 예측
model.predict(X_test)[0]

4/4 [==============================] - 7s 947ms/step


array([[0.29867896],
       [0.1616599 ],
       [0.26786387],
       [0.14012703],
       [0.1454579 ],
       [0.28052437],
       [0.17603055],
       [0.6733469 ],
       [0.2871585 ],
       [0.67560565],
       [0.54533523],
       [0.4897847 ],
       [0.6435701 ],
       [0.3561145 ],
       [0.3323094 ],
       [0.5790015 ],
       [0.48869148],
       [0.4549428 ],
       [0.2599839 ],
       [0.5327864 ],
       [0.4541444 ],
       [0.51526594],
       [0.5905981 ],
       [0.5950458 ],
       [0.55296063],
       [0.5445281 ],
       [0.43073007],
       [0.36903465],
       [0.5227907 ],
       [0.55001324],
       [0.47427756],
       [0.34131622],
       [0.5162238 ],
       [0.49527282],
       [0.4722088 ],
       [0.4370702 ],
       [0.43921188],
       [0.31180292],
       [0.33214867],
       [0.21364255],
       [0.40061247],
       [0.38315246],
       [0.4401995 ],
       [0.4946472 ],
       [0.42315093],
       [0.42934418],
       [0.3321649 ],
       [0.438